In [ ]:
## Change to code if plots do not show up.
%matplotlib inline

# Spatial and Timeseries visualisation of AWRA-L inputs and outputs

The inputs to and results of an AWRA-L simulation can be viewed both spatially and temporally using the tools of the AWRA MS. 



This notebook will demonstrate the available functionality to visualise outputs or inputs that are in the format used by the model, <br>
i.e. in CF compliant netcdf files that contain a year's worth of daily data for a single variable.
<BR>

In addition, it will be shown how other tools can be used for extracting subsets of data from the overall outputs.

The following steps are covered:
    
    1. Import required libraries
    2. Load inputs or results
        2.1 Inspect which variables are available
    3. Spatial plots for different data slices
        3.1 A single day, whole extents
        3.2 Aggregated over a month for specified region
        3.3 Change aggregation method
        3.4 Accessing the underlying data
        3.5 Specifying a catchment extent
        3.6 Show location of catchment
        3.7 Manipulating matplotlib settings
    4. Timeseries plots for selected locations/regions
        4.1 Time series for a single location
        4.2 Time series for a catchment
    5. Exercise

### 1. Import required modules

In [ ]:
import os

from awrams.utils import config_manager, extents
from awrams.utils.gis import ShapefileDB
import awrams.visualisation.vis as vis
import awrams.visualisation.results as res

### 2.  Load inputs or results

expects to find netcdf files in folder

In [ ]:
# NOTE: This notebook depends on results generated by ../Simulation/SimulationServer.ipynb.
# If you haven't already done so, run this notebook before running the following.
sim_output_dir = '../Simulation/results_simserver'
results = res.load_results(sim_output_dir)

sys_settings = config_manager.get_system_profile().get_settings()
climate_training_path = sys_settings['CLIMATE_DATASETS']['TRAINING']['FORCING']['PATH']
inputs = res.load_results(climate_training_path + '/rain_day/')

In [ ]:
results.path

In [ ]:
inputs.path

#### 2.1 Inspect variables present in results
tab completable access to variable list

In [ ]:
results

In [ ]:
results.period

In [ ]:
results.variables.qtot

In [ ]:
inputs.variables

### 3.  Spatial plots for different data slices

##### Approach: define slice of interest for viewing = [variables, period, extent]<br>
variables slice
* display a single variable - results.variables.qtot
* display multiple variables with a tuple - results.variables.qtot,results.variables.ss
* display all variables using standard slicing syntax - ":"

period slice
* a single day - "1 jul 2010"
* a period - "jul 2010" or "jul 2010 - jun 2011"
* data will be aggregated over the period using the specified method (pass aggregate_method='average' or aggregate_method='sum')<br>or the default method for a variable

extent slice
* entire spatial extents - ":" or extents.get_default_extent()
* a bounding box - results.extent.icoords[-39.5:-44,143.5:149]
* a catchment - catchments.get_extent_by_field('GaugeName','Gunning',results.extent)

#### 3.1 A single day, whole extents

In [ ]:
results[:, '12 dec 2010', :].spatial()

In [ ]:
inputs[:, '1 jan 2011', :].spatial() # the entire grid is covered in data because the input file is not masked

In [ ]:
inputs[:, '2 dec 2010', extents.get_default_extent()].spatial()  #  one way to mask it is to specify the default extent

#### 3.2 Aggregated over a month for specified region
Also saves picture to png

In [ ]:
v = results.variables.sd, results.variables.ss

results[v, 'jan 2011', results.extent.icoords[-41:-44, 143.5:149]].spatial()

vis.plt.savefig('map_of_tasmania.png', format='png', dpi=120)

#### 3.3. Change aggregation method 

Here, from default of mean to sum

In [ ]:
v

In [ ]:
v = results.variables.qtot
v.agg_method = 'mean'
results[v, 'jan 2011', results.extent.icoords[-39.5:-44, 143.5:149]].spatial()

#### 3.4 Accessing the underlying data
raw data cube

In [ ]:
ss_data = results.variables.ss.data

Data shape

In [ ]:
results.variables.ss.data.shape

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(ss_data[30])
plt.colorbar()

temporally aggregated data

In [ ]:
results.variables.ss.agg_data.shape

In [ ]:
ss_agg_data = results.variables.ss.agg_data
plt.imshow(ss_agg_data)
plt.colorbar()

#### 3.5 Specifying a catchment extent

In [ ]:
## A quick way to inspect the contents of the shapefile
base_data_path = sys_settings['DATA_PATHS']['BASE_DATA']
catchment_shapefile = os.path.join(base_data_path, 'spatial/shapefiles/Final_list_all_attributes.shp')

default_catchments = ShapefileDB(catchment_shapefile)
default_catchments_df = default_catchments.get_records_df()
default_catchments_df .head()

In [ ]:
gunning = default_catchments.get_extent_by_field('GaugeName', 'Gunning', results.extent)
gunning

In [ ]:
v = results.variables.ss, results.variables.sd
results[v,'jan 2011',gunning].spatial(interpolation=None) #interpolation="bilinear")

#### 3.6 Show location of catchment

In [ ]:
#vis.show_extent(gunning)
vis.show_extent(gunning, results.extent.icoords[-30:-40,142:154])
vis.show_extent(gunning, results.extent)
vis.show_extent(gunning, gunning)

#### 3.7 Manipulating matplotlib settings

##### Specifying plotting ranges and labelling

In [ ]:
v = results.variables.qtot, results.variables.ss
results[v,'jan 2011',:].spatial(clim=(0, 100), xlabel="longitude")

#### accessing the underlying axes
get the range of data for the selection

In [ ]:
q = results[v,'jan 2011', results.extent.icoords[-30:-40,142:154]]
q.get_data_limits()

set colour range limits and horizontal axis labels

In [ ]:
q.spatial(clim=(0, 200), xlabel="longitude")

gridview = q.mpl
view = gridview.children[0, 1]

view.ax.set_xlabel("ALSO LONGITUDE!")
vis.plt.show()

### 4. Timeseries plots for selected locations/regions

#### 4.1 Time series for a single location

In [ ]:
p = 'jan 2011'
e = results.extent.icoords[-34, 117]
results[:,p,e].timeseries()

#### 4.2 Time series of aggregate over catchment

By default, the average over a catchment will be produced if you provide a catchment extent

In [ ]:
edef = results.extent

In [ ]:
v = results.variables.qtot,results.variables.ss
p = 'jan 2011'
e = default_catchments.get_extent_by_field('GaugeName', 'Mittagang Crossing', results.extent)
results[v,p,e].timeseries()

In [ ]:
results.variables.qtot.data.shape,results.variables.qtot.agg_data.shape

#### 4.3 More Matplotlib formatting options

In [ ]:
v = results.variables.qtot
e = results.extent.icoords[-34, 117]
p = 'jan 2011'

q = results[v,p,e]
q.timeseries()

In [ ]:
q.timeseries() #ylim=(0,20))  # q is an object that carries data as well as matplotlib configurations

ax = q.mpl.children[0,0].ax
lines = ax.get_lines()
line = lines[0]
line.set_color('m')
leg = ax.get_legend()
legline = leg.get_lines()[0]
legline.set_color('m')

### 5. Exercise: Visualising climatic inputs


In [ ]:
sys_settings = config_manager.get_system_profile().get_settings()
climate_training_path = sys_settings['CLIMATE_DATASETS']['TRAINING']['FORCING']['PATH']

In [ ]:
precip = res.load_results(climate_training_path + '/rain_day/')

vis.spatial(precip.variables.rain_day, period='16 dec 2010')

## If you want to see the data only over Australia, use the defaul extent definition
vis.spatial(precip.variables.rain_day, period='16 dec 2010', extent=extents.get_default_extent())

In [ ]:
tmax = res.load_results(climate_training_path + '/temp_max_day/')

vis.spatial(tmax.variables.temp_max_day, period='16 may 2010', extent=extents.get_default_extent())

In [ ]:
tmin = res.load_results(climate_training_path  + '/temp_min_day/')
vis.spatial(tmin.variables.temp_min_day, period='16 may 2010', extent=extents.get_default_extent())

In [ ]:
solar = res.load_results(climate_training_path + '/solar_exposure_day/')
vis.spatial(solar.variables.solar_exposure_day, period='16 may 2010')

In [ ]:
wind = res.load_results(climate_training_path + '/wind/')
vis.spatial(wind.variables.wind, period='16 may 2010')